In [ ]:

import os
import numpy as np
import rospy
import rosbag
import json
import message_filters # 仅用于 Duration

# ROS 消息和工具
from cv_bridge import CvBridge
from sensor_msgs.msg import Image, CameraInfo
import cv2
# --- [!!! 用户配置 !!!] ---
# 1. ROS 话题 (根据用户提供的 Bag Info)
TOPIC_RGB = '/camera/color/image_raw' 
TOPIC_DEPTH = '/camera/aligned_depth_to_color/image_raw' 
TOPIC_INFO = '/camera/color/camera_info' 

# 2. 目标帧配置
BAG_FILE_PATH = '../2025-12-01-19-01-56.bag'
TARGET_FRAME_INDEX = 50 # 想要处理的同步帧索引 (例如第 50 帧)
OUTPUT_DIR = "processed_frame_data"
# -----------------------------

class FrameExtractor:
    """
    离线处理 ROS Bag 文件，提取指定帧的 RGB, 深度, 和相机内参。
    """
    def __init__(self):
        self.bridge = CvBridge()

    def _save_data(self, rgb_img, depth_img, info_msg, frame_index, output_dir):
        """保存图像数据和内参信息。"""
        os.makedirs(output_dir, exist_ok=True)
        
        base_name = f"frame_{frame_index}"
        
        # 保存 RGB 图像 (PNG)
        rgb_filepath = os.path.join(output_dir, f"{base_name}_rgb.png")
        cv2.imwrite(rgb_filepath, rgb_img)
        rospy.loginfo(f" -> 已保存 RGB 图像: {rgb_filepath}")
        
        # 保存原始深度数据 (NumPy 数组)
        depth_filepath = os.path.join(output_dir, f"{base_name}_depth.npy")
        np.save(depth_filepath, depth_img)
        rospy.loginfo(f" -> 已保存深度数据: {depth_filepath}")
        
        # 保存相机内参 K (JSON 格式)
        info_data = {
            'K': list(info_msg.K),
            'header': {'stamp': info_msg.header.stamp.to_sec(), 'frame_id': info_msg.header.frame_id}
        }
        info_filepath = os.path.join(output_dir, f"{base_name}_info.json")
        with open(info_filepath, 'w') as f:
            json.dump(info_data, f, indent=4)
        rospy.loginfo(f" -> 已保存相机内参: {info_filepath}")
        
        rospy.loginfo(f"--- 帧 {frame_index} 数据保存成功 ---")

    def run_offline(self, bag_path, target_index):
        """Offline processing loop to find and process the target frame."""
        rospy.loginfo(f"开始离线处理。目标帧索引: {target_index}，输出目录: {OUTPUT_DIR}")
        
        frame_counter = 0
        
        try:
            bag = rosbag.Bag(bag_path, 'r')
        except Exception as e:
            rospy.logerr(f"打开 Bag 文件失败: {e}")
            return
        
        topics_to_read = [TOPIC_RGB, TOPIC_DEPTH, TOPIC_INFO]
        TIME_TOLERANCE = rospy.Duration(0.01) # 10ms 容忍度
        time_buffer = {} 

        try:
            for topic, msg, t in bag.read_messages(topics=topics_to_read):
                
                # 收集消息到缓冲区
                time_buffer[topic] = (msg, t)
                
                # 检查是否收集齐了所有三个话题
                if all(t in time_buffer for t in [TOPIC_RGB, TOPIC_DEPTH, TOPIC_INFO]):
                    
                    rgb_t = time_buffer[TOPIC_RGB][1]
                    depth_t = time_buffer[TOPIC_DEPTH][1]
                    info_t = time_buffer[TOPIC_INFO][1]
                    
                    # 检查时间戳是否足够接近
                    if abs(rgb_t - depth_t) < TIME_TOLERANCE and abs(rgb_t - info_t) < TIME_TOLERANCE:
                        
                        frame_counter += 1
                        
                        if frame_counter == target_index:
                            rospy.loginfo(f"--- 找到目标帧 {target_index}，时间: {rgb_t.to_sec()} ---")
                            
                            # 转换 ROS 消息到 OpenCV/NumPy
                            cv_rgb = self.bridge.imgmsg_to_cv2(time_buffer[TOPIC_RGB][0], "bgr8")
                            cv_depth = self.bridge.imgmsg_to_cv2(time_buffer[TOPIC_DEPTH][0], "16UC1")
                            
                            # 保存数据
                            self._save_data(
                                cv_rgb, 
                                cv_depth, 
                                time_buffer[TOPIC_INFO][0], 
                                target_index, 
                                OUTPUT_DIR
                            )
                            return
                        
                        # 清空缓冲区，寻找下一组同步消息
                        time_buffer = {} 
                        
        finally:
            bag.close()
            
        if frame_counter < target_index:
            rospy.logwarn(f"Bag 文件结束。只找到 {frame_counter} 帧同步帧。目标索引 {target_index} 未找到。")
        else:
            rospy.loginfo("离线处理完成。")


if __name__ == '__main__':
    try:
        rospy.init_node('mmseg_frame_extractor', anonymous=True, log_level=rospy.INFO, disable_signals=True)
    except rospy.ROSException:
        pass 

    try:
        extractor = FrameExtractor()
        extractor.run_offline(BAG_FILE_PATH, TARGET_FRAME_INDEX)

    except Exception as e:
        rospy.logerr(f"程序发生严重错误: {e}")

In [ ]:
#!/home/mei123/miniconda3/bin/python
# -*- coding: utf-8 -*-
import os
import struct
import numpy as np
import cv2
import json
import torch 
import rospy
from mmseg.apis import init_model, inference_model 
from mmengine.config import Config
from mmseg.models import BaseSegmentor

# --- [!!! 用户必改配置 !!!] ---
# 1. MMSegmentation 模型路径和配置
CONFIG_FILE = '/home/mei123/workspace/cv/mmsegmentation/config/deeplabv3plus_r50-d8_4xb4-80k_ade20k-512x512.py'
CHECKPOINT_FILE = '/home/mei123/workspace/cv/mmsegmentation/config/deeplabv3plus_r50-d8_512x512_80k_ade20k_20200614_185028-bf1400d8.pth'
 
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# 2. 数据源配置 (必须与 1_extract_frame.py 的输出对应)
TARGET_FRAME_INDEX = 50 
INPUT_DIR = "processed_frame_data"

# 3. 颜色映射 (已删除手动配置，改为运行时自动生成)
# -----------------------------

class SemanticAnnotator:
    """
    加载离线数据，运行 MMSegmentator，并生成语义点云。
    """
    def __init__(self):
        # 如果在交互式环境中运行，节点可能已经初始化，这里跳过
        if not rospy.core.is_initialized():
            rospy.init_node('mmseg_annotator', anonymous=True, log_level=rospy.INFO)
        else:
            rospy.loginfo("ROS node already initialized, continuing...")
            
        self.model = self._init_mmseg_model()
        # 运行时缓存已生成的颜色，避免重复计算
        self.color_cache = {} 
        self.class_id_map = {} 


    def _init_mmseg_model(self):
        """初始化 MMSegmentation 模型。"""
        try:
            model = init_model(CONFIG_FILE, CHECKPOINT_FILE, device=DEVICE)
            model.eval()
            rospy.loginfo(f"MMSegmentation 模型加载成功，运行在: {DEVICE}")
            return model
        except Exception as e:
            rospy.logerr(f"模型加载失败: {e}")
            return None

    def _generate_unique_color(self, class_id: int) -> tuple:
        """
        根据类别 ID 自动生成一个独特且可区分的 RGB 颜色。
        """
        # 0: 背景，强制设为黑色
        if class_id == 0:
            return (0, 0, 0)
        
        # 检查缓存
        if class_id in self.color_cache:
            return self.color_cache[class_id]

        # 使用固定算法生成颜色，确保颜色一致性
        r = (class_id * 37 + 50) % 256
        g = (class_id * 101 + 100) % 256
        b = (class_id * 179 + 150) % 256
        
        color = (r, g, b)
        self.color_cache[class_id] = color
        # 记录 ID 到 RGB 列表的映射，方便 JSON 序列化
        self.class_id_map[class_id] = list(color) 
        return color

    def _load_data(self, frame_index, input_dir):
        """加载保存的 RGB, 深度, 和内参文件。"""
        base_name = f"frame_{frame_index}"
        
        rgb_filepath = os.path.join(input_dir, f"{base_name}_rgb.png")
        depth_filepath = os.path.join(input_dir, f"{base_name}_depth.npy")
        info_filepath = os.path.join(input_dir, f"{base_name}_info.json")

        if not all(os.path.exists(f) for f in [rgb_filepath, depth_filepath, info_filepath]):
            rospy.logerr(f"错误: 找不到帧 {frame_index} 的所有输入文件。请先运行 1_extract_frame.py。")
            return None, None, None, None
            
        rgb_img = cv2.imread(rgb_filepath) # BGR 格式
        depth_img = np.load(depth_filepath) # 16UC1 格式
        
        with open(info_filepath, 'r') as f:
            info_data = json.load(f)
            K_matrix = info_data['K']
            frame_id = info_data['header']['frame_id']

        rospy.loginfo(f"数据加载成功。RGB形状: {rgb_img.shape}, 深度形状: {depth_img.shape}")
        
        # 提取 K=[fx, fy, cx, cy]
        K = [K_matrix[0], K_matrix[4], K_matrix[2], K_matrix[5]]
        
        return rgb_img, depth_img, K, frame_id

    def _save_pcd_file(self, data: np.ndarray, filename: str, output_dir: str):
        """手动将 NumPy 数组写入 ASCII PCD 文件。"""
        filepath = os.path.join(output_dir, filename)
        num_points = len(data)
        fields = data.dtype.names
        
        header = [
            '# .PCD v0.7 - Point Cloud Data file format', 'VERSION 0.7',
            f'FIELDS {" ".join(fields)}', f'SIZE 4 4 4 4', 
            f'TYPE F F F U', f'COUNT 1 1 1 1',
            f'WIDTH {num_points}', 'HEIGHT 1',
            'VIEWPOINT 0 0 0 1 0 0 0', f'POINTS {num_points}', 'DATA ascii'
        ]
        
        with open(filepath, 'w') as f:
            f.write('\n'.join(header) + '\n')
            for row in data:
                # RGB 字段存储为 UINT32 整数
                rgb_hex = int(row['rgb'].item())
                f.write(f"{row['x']} {row['y']} {row['z']} {rgb_hex}\n")

        rospy.loginfo(f" -> 已保存点云文件: {filepath}")
    
    def _encode_rgb_to_uint32(self, r: int, g: int, b: int) -> int:
        """将 R, G, B 整数分量安全编码为 UINT32 整数 (BGR 序)。"""
        # 使用位运算，确保返回Python int
        return (int(b) << 16) | (int(g) << 8) | int(r)

    def _extract_pixel_color_safe(self, rgb_img: np.ndarray, v: int, u: int) -> tuple:
        """安全地提取单个像素的BGR颜色值，确保返回Python int类型。"""
        # 直接使用整数索引和转换
        b = int(rgb_img[v, u, 0])
        g = int(rgb_img[v, u, 1])
        r = int(rgb_img[v, u, 2])
        return r, g, b

    def run_annotation(self):
        """执行整个标注和保存流程。"""
        if self.model is None: return

        # 1. 加载数据
        rgb_img, depth_img, K, frame_id = self._load_data(TARGET_FRAME_INDEX, INPUT_DIR)
        if rgb_img is None: return

        # 2. MMSegmentation 推理
        rospy.loginfo("开始 MMSegmentation 推理...")
        semantic_mask = self._get_semantic_mask(rgb_img)
        rospy.loginfo("语义分割完成。开始点云融合...")
        
        # 3. 点云生成与融合
        fx, fy, cx, cy = K
        H, W = semantic_mask.shape
        DEPTH_SCALE = 1000.0 # RealSense is typically mm

        points_original = []
        points_semantic = []

        # 预分配数组以提高性能
        for v in range(H):
            for u in range(W):
                # 检查深度，确保它是一个可操作的整数类型
                depth_mm = depth_img[v, u]
                if depth_mm == 0: 
                    continue
                
                # 转换为Python标量值
                depth_val = float(depth_mm)
                Z = depth_val / DEPTH_SCALE
                X = (u - cx) * Z / fx
                Y = (v - cy) * Z / fy
                
                # --- 原始 RGB 颜色 (使用安全提取方法) ---
                r_orig, g_orig, b_orig = self._extract_pixel_color_safe(rgb_img, v, u)
                rgb_int_orig = self._encode_rgb_to_uint32(r_orig, g_orig, b_orig)
                
                # --- 语义颜色 (自动生成) ---
                class_id = int(semantic_mask[v, u])
                r_sem, g_sem, b_sem = self._generate_unique_color(class_id)
                rgb_int_sem = self._encode_rgb_to_uint32(r_sem, g_sem, b_sem)

                # 添加到点云列表
                points_original.append((X, Y, Z, rgb_int_orig))
                points_semantic.append((X, Y, Z, rgb_int_sem))

        dtype = np.dtype([('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('rgb', 'u4')])

        # 4. 保存点云文件
        if points_original:
            points_original_np = np.array(points_original, dtype=dtype)
            self._save_pcd_file(points_original_np, f"frame_{TARGET_FRAME_INDEX}_original_rgb.pcd", INPUT_DIR)
        else:
            rospy.logwarn("原始点云为空，跳过保存")
            
        if points_semantic:
            points_semantic_np = np.array(points_semantic, dtype=dtype)
            self._save_pcd_file(points_semantic_np, f"frame_{TARGET_FRAME_INDEX}_semantic.pcd", INPUT_DIR)
        else:
            rospy.logwarn("语义点云为空，跳过保存")
        
        map_filepath = os.path.join(INPUT_DIR, f"frame_{TARGET_FRAME_INDEX}_semantic_map.json")
        with open(map_filepath, 'w') as f:
            # JSON 键必须是字符串，因此将整数键转换为字符串
            serializable_map = {str(k): v for k, v in self.class_id_map.items()}
            json.dump(serializable_map, f, indent=4)
        rospy.loginfo(f" -> 已保存语义颜色映射表: {map_filepath}")
        
        rospy.loginfo(f"--- 语义点云生成和保存完成 ---")
        rospy.loginfo(f"原始点云点数: {len(points_original)}")
        rospy.loginfo(f"语义点云点数: {len(points_semantic)}")

    def _get_semantic_mask(self, cv_image: np.ndarray) -> np.ndarray:
        """运行模型推理并获取语义分割掩码 (H, W)。"""
        result = inference_model(self.model, cv_image)
        semantic_mask = result.pred_sem_seg.data.cpu().numpy().squeeze()
        return semantic_mask


if __name__ == '__main__':
    try:
        annotator = SemanticAnnotator()
        annotator.run_annotation()

    except Exception as e:
        rospy.logerr(f"程序发生严重错误: {e}")
        import traceback
        rospy.logerr(traceback.format_exc())

In [ ]:
# #!/home/mei123/miniconda3/bin/python
# # -*- coding: utf-8 -*-
# import os
# import struct
# import numpy as np
# import rospy
# import time

# # ROS 消息和工具
# from sensor_msgs.msg import PointCloud2, PointField
# import sensor_msgs.point_cloud2 as pc2

# # --- [!!! 用户配置 !!!] ---
# TARGET_FRAME_INDEX = 50 
# INPUT_DIR = "processed_frame_data"
# # 要发布的 PCD 文件名 (请确保运行 2_annotate_pcd.py 后该文件存在)
# PCD_FILENAME = f"frame_{TARGET_FRAME_INDEX}_semantic.pcd" 
# # ROS 话题名称
# PUBLISH_TOPIC = '/processed/semantic_point_cloud'
# # PCD 文件的 Frame ID
# FRAME_ID = 'camera_link' 
# # -----------------------------

# class PCDPublisher:
#     """
#     读取 PCD 文件并持续发布到 ROS 话题。
#     """
#     def __init__(self):
#         # 如果在交互式环境中运行，节点可能已经初始化，这里跳过
#         if not rospy.core.is_initialized():
#             rospy.init_node('mmseg_annotator', anonymous=True, log_level=rospy.INFO)
#         else:
#             rospy.loginfo("ROS node already initialized, continuing...")
#         self.pub = rospy.Publisher(PUBLISH_TOPIC, PointCloud2, queue_size=1)
#         self.pcd_data_np = self._load_pcd_data()

#     def _load_pcd_data(self):
#         """手动读取 ASCII PCD 文件中的点云数据。"""
#         filepath = os.path.join(INPUT_DIR, PCD_FILENAME)
        
#         if not os.path.exists(filepath):
#             rospy.logerr(f"错误: 找不到 PCD 文件: {filepath}. 请确保已运行 2_annotate_pcd.py。")
#             rospy.signal_shutdown("PCD 文件缺失")
#             return None

#         rospy.loginfo(f"加载 PCD 文件: {filepath}")
        
#         # 读取 PCD 文件 (简单 ASCII 格式)
#         data = []
#         with open(filepath, 'r') as f:
#             # 跳过头部，直到 'DATA ascii'
#             is_data_section = False
#             for line in f:
#                 line = line.strip()
#                 if line.startswith('DATA'):
#                     is_data_section = True
#                     continue
#                 if is_data_section and line:
#                     try:
#                         # 格式: x y z rgb_uint32
#                         parts = line.split()
#                         x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
#                         rgb_int = int(parts[3])
#                         data.append((x, y, z, rgb_int))
#                     except ValueError:
#                         rospy.logwarn(f"跳过 PCD 数据行: {line}")
#                         continue
        
#         rospy.loginfo(f"成功加载 {len(data)} 个点。")
        
#         # 创建 NumPy 结构数组
#         dtype = np.dtype([('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('rgb', 'u4')])
#         return np.array(data, dtype=dtype)

#     def publish_loop(self):
#         """持续发布点云消息。"""
#         if self.pcd_data_np is None: return

#         # 定义 PointCloud2 字段
#         fields = [
#             PointField('x', 0, PointField.FLOAT32, 1),
#             PointField('y', 4, PointField.FLOAT32, 1),
#             PointField('z', 8, PointField.FLOAT32, 1),
#             PointField('rgb', 12, PointField.UINT32, 1)
#         ]
        
#         rate = rospy.Rate(1) # 每秒发布一次
#         rospy.loginfo(f"开始发布点云到话题 {PUBLISH_TOPIC}")

#         while not rospy.is_shutdown():
#             header = rospy.Header()
#             header.stamp = rospy.Time.now()
#             header.frame_id = FRAME_ID
            
#             # 创建 PointCloud2 消息
#             pc_msg = pc2.create_cloud(header, fields, self.pcd_data_np.tolist())
            
#             self.pub.publish(pc_msg)
#             rospy.loginfo(f"已发布点云 ({self.pcd_data_np.shape[0]} 点)")
            
#             rate.sleep()


# if __name__ == '__main__':
#     try:
#         publisher = PCDPublisher()
#         publisher.publish_loop()
#     except rospy.ROSInterruptException:
#         rospy.loginfo("ROS 中断，退出。")
#!/home/mei123/miniconda3/bin/python
# -*- coding: utf-8 -*-
import os
import struct
import numpy as np
import rospy
import time
import tf2_ros
from geometry_msgs.msg import TransformStamped

# ROS 消息和工具
from sensor_msgs.msg import PointCloud2, PointField
import sensor_msgs.point_cloud2 as pc2

# --- [!!! 用户配置 !!!] ---
TARGET_FRAME_INDEX = 50 
INPUT_DIR = "processed_frame_data"
# 要发布的 PCD 文件名 (请确保运行 2_annotate_pcd.py 后该文件存在)
PCD_FILENAME = f"frame_{TARGET_FRAME_INDEX}_semantic.pcd" 
PCD_FILENAME = f"frame_{TARGET_FRAME_INDEX}_original_rgb.pcd" 

# ROS 话题名称
PUBLISH_TOPIC = '/processed/semantic_point_cloud'
# PCD 文件的 Frame ID (已更新为用户指定值)
FRAME_ID = 'camera_color_frame' 
# 全局固定坐标系 ID
MAP_FRAME_ID = 'map'
# -----------------------------

class PCDPublisher:
    """
    读取 PCD 文件并持续发布到 ROS 话题，同时广播 TF 关系。
    """
    def __init__(self):
        # 如果在交互式环境中运行，节点可能已经初始化，这里跳过
        if not rospy.core.is_initialized():
            rospy.init_node('mmseg_annotator', anonymous=True, log_level=rospy.INFO)
        else:
            rospy.loginfo("ROS node already initialized, continuing...")
        self.pub = rospy.Publisher(PUBLISH_TOPIC, PointCloud2, queue_size=1)
        
        # 初始化 TF 广播器
        self.tf_broadcaster = tf2_ros.TransformBroadcaster()
        
        self.pcd_data_np = self._load_pcd_data()

    def _load_pcd_data(self):
        """手动读取 ASCII PCD 文件中的点云数据。"""
        filepath = os.path.join(INPUT_DIR, PCD_FILENAME)
        
        if not os.path.exists(filepath):
            rospy.logerr(f"错误: 找不到 PCD 文件: {filepath}. 请确保已运行 2_annotate_pcd.py。")
            rospy.signal_shutdown("PCD 文件缺失")
            return None

        rospy.loginfo(f"加载 PCD 文件: {filepath}")
        
        # 读取 PCD 文件 (简单 ASCII 格式)
        data = []
        with open(filepath, 'r') as f:
            # 跳过头部，直到 'DATA ascii'
            is_data_section = False
            for line in f:
                line = line.strip()
                if line.startswith('DATA'):
                    is_data_section = True
                    continue
                if is_data_section and line:
                    try:
                        # 格式: x y z rgb_uint32
                        parts = line.split()
                        x, y, z = float(parts[0]), float(parts[1]), float(parts[2])
                        rgb_int = int(parts[3])
                        data.append((x, y, z, rgb_int))
                    except ValueError:
                        rospy.logwarn(f"跳过 PCD 数据行: {line}")
                        continue
        
        rospy.loginfo(f"成功加载 {len(data)} 个点。")
        
        # 创建 NumPy 结构数组
        dtype = np.dtype([('x', 'f4'), ('y', 'f4'), ('z', 'f4'), ('rgb', 'u4')])
        return np.array(data, dtype=dtype)

    def publish_loop(self):
        """持续发布点云消息和 TF 变换。"""
        if self.pcd_data_np is None: return

        # 定义 PointCloud2 字段
        fields = [
            PointField('x', 0, PointField.FLOAT32, 1),
            PointField('y', 4, PointField.FLOAT32, 1),
            PointField('z', 8, PointField.FLOAT32, 1),
            PointField('rgb', 12, PointField.UINT32, 1)
        ]
        
        rate = rospy.Rate(1) # 每秒发布一次
        rospy.loginfo(f"开始发布点云到话题 {PUBLISH_TOPIC}")

        while not rospy.is_shutdown():
            current_time = rospy.Time.now()

            # --- 1. 广播 TF 变换 (从 map 到 camera_color_frame) ---
            t = TransformStamped()
            t.header.stamp = current_time
            t.header.frame_id = MAP_FRAME_ID # 父坐标系 (全局)
            t.child_frame_id = FRAME_ID      # 子坐标系 (点云坐标系)
            
            # 假设一个简单的零位姿变换，将相机原点置于 map 原点
            t.transform.translation.x = 0.0
            t.transform.translation.y = 0.0
            t.transform.translation.z = 0.0
            t.transform.rotation.x = 0.0
            t.transform.rotation.y = 0.0
            t.transform.rotation.z = 0.0
            t.transform.rotation.w = 1.0 # 零旋转
            
            self.tf_broadcaster.sendTransform(t)
            
            # --- 2. 发布 PointCloud2 消息 ---
            header = rospy.Header()
            header.stamp = current_time
            header.frame_id = FRAME_ID # 使用 camera_color_frame 作为点云的参考系
            
            # 创建 PointCloud2 消息
            pc_msg = pc2.create_cloud(header, fields, self.pcd_data_np.tolist())
            
            self.pub.publish(pc_msg)
            rospy.loginfo(f"已发布点云 ({self.pcd_data_np.shape[0]} 点) 和 TF 变换。")
            
            rate.sleep()


if __name__ == '__main__':
    try:
        publisher = PCDPublisher()
        publisher.publish_loop()
    except rospy.ROSInterruptException:
        rospy.loginfo("ROS 中断，退出。")

In [3]:
#!/home/mei123/miniconda3/bin/python
# -*- coding: utf-8 -*-
import os
import json
import rospy
from mmengine.config import Config
from prettytable import PrettyTable # 用于美观地打印表格

# --- [!!! 用户配置 !!!] ---
# 必须与 2_annotate_pcd.py 中的配置一致
CONFIG_FILE = '/home/mei123/workspace/cv/mmsegmentation/config/deeplabv3plus_r50-d8_4xb4-80k_ade20k-512x512.py' 
TARGET_FRAME_INDEX = 50 
INPUT_DIR = "processed_frame_data"
# -----------------------------

class SemanticAnalyzer:
    """
    分析 MMSegmentation 配置和运行时生成的颜色映射表，输出最终的语义对应关系。
    """
    def __init__(self):
        if not rospy.core.is_initialized():
            rospy.init_node('mmseg_analyzer', anonymous=True, log_level=rospy.INFO)
        else:
            rospy.loginfo("ROS node already initialized, continuing...")

    def _load_model_classes(self):
        """从 MMSegmentation 配置文件中加载类别名称列表。"""
        try:
            cfg = Config.fromfile(CONFIG_FILE)
            class_names = None
            
            # 1. 尝试从 metainfo 中获取 (MMSeg 1.x 标准)
            if 'metainfo' in cfg and 'classes' in cfg.metainfo:
                class_names = cfg.metainfo.classes
                rospy.loginfo(f"从 metainfo 加载 {len(class_names)} 个类别名称 (MMSeg 1.x)。")
                
            # 2. 兼容旧配置 (MMSeg 0.x)
            elif 'classes' in cfg:
                 class_names = cfg.classes
                 rospy.loginfo(f"从 classes 字段加载 {len(class_names)} 个类别名称 (MMSeg 0.x)。")

            # 3. 如果配置中没有，使用 Registry 查找 Dataset 类 (最鲁棒的方法)
            else:
                if 'train_dataloader' in cfg and 'dataset' in cfg.train_dataloader and 'type' in cfg.train_dataloader.dataset:
                    dataset_type_str = cfg.train_dataloader.dataset.type
                    rospy.loginfo(f"配置文件未指定 metainfo.classes，尝试从 Dataset Registry '{dataset_type_str}' 中查找。")
                    
                    try:
                        # 引入 MMSegmentation 官方 Registry
                        from mmseg.registry import DATASETS
                        
                        DatasetClass = DATASETS.get(dataset_type_str)

                        if DatasetClass and hasattr(DatasetClass, 'METAINFO') and 'classes' in DatasetClass.METAINFO:
                            # MMSeg 1.x 标准：从 METAINFO 字典中获取
                            class_names = DatasetClass.METAINFO['classes']
                            rospy.loginfo(f"成功从 Registry/METAINFO 中获取 {len(class_names)} 个类别名称。")
                        elif DatasetClass and hasattr(DatasetClass, 'CLASSES'):
                            # Fallback：从 CLASSES 属性中获取 (兼容 0.x)
                            class_names = DatasetClass.CLASSES
                            rospy.loginfo(f"成功从 Registry/CLASSES 属性中获取 {len(class_names)} 个类别名称。")
                        else:
                            rospy.logerr(f"错误: 无法从 Dataset Registry 中获取 CLASSES/METAINFO 属性。")
                            return None
                    except Exception as registry_e:
                        rospy.logerr(f"尝试使用 Registry 查找 Dataset 失败: {registry_e}")
                        return None
                else:
                    rospy.logerr("错误: 配置文件中找不到类别名称 (classes) 且无法推断数据集类型。请检查 CONFIG_FILE 路径和内容。")
                    return None
                 
            # --- 关键修复：强制转换 class_names 为标准 Python 列表 ---
            if class_names is not None:
                try:
                    class_names = list(class_names)
                except Exception as e:
                    rospy.logerr(f"错误: 无法将获取到的类别对象转换为列表: {e}")
                    return None
                    
            # 通用检查 (如果转换后仍失败，则说明数据有问题)
            if not isinstance(class_names, list) or len(class_names) == 0:
                 rospy.logerr("错误: 类别列表为空或格式不正确。")
                 return None

            # 创建 ID -> Name 的映射，ID 从 0 开始
            id_to_name = {i: name for i, name in enumerate(class_names)}
            return id_to_name

        except Exception as e:
            rospy.logerr(f"加载模型配置文件失败: {e}")
            rospy.logerr("请确认已安装 mmseg 和 mmengine 且 CONFIG_FILE 路径正确。")
            return None

    def _load_color_map(self):
        """加载由 2_annotate_pcd.py 生成的 ID -> RGB 映射表。"""
        map_filepath = os.path.join(INPUT_DIR, f"frame_{TARGET_FRAME_INDEX}_semantic_map.json")
        
        if not os.path.exists(map_filepath):
            rospy.logerr(f"错误: 找不到颜色映射表文件: {map_filepath}。请先运行 2_annotate_pcd.py。")
            return None

        try:
            with open(map_filepath, 'r') as f:
                # JSON 键是字符串，需要转换回整数 ID
                color_map_str_keys = json.load(f)
                id_to_color = {int(k): tuple(v) for k, v in color_map_str_keys.items()}
                rospy.loginfo(f"加载 {len(id_to_color)} 个类别 ID 的颜色映射。")
                return id_to_color
        except Exception as e:
            rospy.logerr(f"加载或解析颜色映射表失败: {e}")
            return None

    def run_analysis(self):
        """执行分析并打印结果表格。"""
        id_to_name = self._load_model_classes()
        id_to_color = self._load_color_map()

        if id_to_name is None or id_to_color is None:
            rospy.logerr("无法继续分析，缺少关键数据。")
            return

        # 创建表格
        table = PrettyTable()
        table.field_names = ["ID", "Semantic Name (语义标签)", "RGB Color (点云颜色)"]
        table.align["Semantic Name (语义标签)"] = "l"
        
        rospy.loginfo("\n--- 语义点云颜色映射表 ---")
        
        # 遍历所有已使用的类别 ID (以颜色映射为准)
        sorted_ids = sorted(id_to_color.keys())
        
        for class_id in sorted_ids:
            color_rgb = id_to_color.get(class_id, (0, 0, 0))
            # 从模型配置中获取名称
            class_name = id_to_name.get(class_id, "UNKNOWN/未定义")
            
            rgb_str = f"({color_rgb[0]}, {color_rgb[1]}, {color_rgb[2]})"
            
            table.add_row([class_id, class_name, rgb_str])

        print(table)
        rospy.loginfo("\n--- 分析完成。请使用此表格在 RViz 中识别语义标签。---\n")


if __name__ == '__main__':
    try:
        analyzer = SemanticAnalyzer()
        analyzer.run_analysis()
    except Exception as e:
        rospy.logerr(f"分析器发生错误: {e}")

[INFO] [1764752538.411415]: ROS node already initialized, continuing...
[INFO] [1764752538.422457]: 配置文件未指定 metainfo.classes，尝试从 Dataset Registry 'ADE20KDataset' 中查找。
[INFO] [1764752538.423621]: 成功从 Registry/METAINFO 中获取 150 个类别名称。
[INFO] [1764752538.424477]: 加载 11 个类别 ID 的颜色映射。
[INFO] [1764752538.425114]: 
--- 语义点云颜色映射表 ---
+-----+--------------------------+----------------------+
|  ID | Semantic Name (语义标签) | RGB Color (点云颜色) |
+-----+--------------------------+----------------------+
|  3  | floor                    |   (161, 147, 175)    |
|  10 | cabinet                  |    (164, 86, 148)    |
|  12 | person                   |    (238, 32, 250)    |
|  14 | door                     |    (56, 234, 96)     |
|  20 | car                      |    (22, 72, 146)     |
|  24 | shelf                    |    (170, 220, 94)    |
|  41 | box                      |    (31, 145, 65)     |
|  50 | refrigerator             |    (108, 30, 140)    |
| 107 | washer                   |   (169, 

[INFO] [1764752558.568344]: 
--- 定时触发 (50 秒)，开始语义处理 ---


[ERROR] [1764752558.569618]: 定时处理时发生错误: 'SemanticProcessorNode' object has no attribute 'bridge'
[ERROR] [1764752558.570705]: Traceback (most recent call last):
  File "/tmp/ipykernel_13511/2187352137.py", line 192, in processing_timer_callback
    cv_rgb = self.bridge.imgmsg_to_cv2(self.latest_rgb_msg, "bgr8")
AttributeError: 'SemanticProcessorNode' object has no attribute 'bridge'



[INFO] [1764752608.548792]: 
--- 定时触发 (50 秒)，开始语义处理 ---


[ERROR] [1764752608.550028]: 定时处理时发生错误: 'SemanticProcessorNode' object has no attribute 'bridge'
[ERROR] [1764752608.550777]: Traceback (most recent call last):
  File "/tmp/ipykernel_13511/2187352137.py", line 192, in processing_timer_callback
    cv_rgb = self.bridge.imgmsg_to_cv2(self.latest_rgb_msg, "bgr8")
AttributeError: 'SemanticProcessorNode' object has no attribute 'bridge'



...

[ERROR] [1764752708.565531]: 定时处理时发生错误: 'SemanticProcessorNode' object has no attribute 'bridge'
[ERROR] [1764752708.566258]: Traceback (most recent call last):
  File "/tmp/ipykernel_13511/2187352137.py", line 192, in processing_timer_callback
    cv_rgb = self.bridge.imgmsg_to_cv2(self.latest_rgb_msg, "bgr8")
AttributeError: 'SemanticProcessorNode' object has no attribute 'bridge'



[INFO] [1764752758.566564]: 
--- 定时触发 (50 秒)，开始语义处理 ---


[ERROR] [1764752758.567698]: 定时处理时发生错误: 'SemanticProcessorNode' object has no attribute 'bridge'
[ERROR] [1764752758.568631]: Traceback (most recent call last):
  File "/tmp/ipykernel_13511/2187352137.py", line 192, in processing_timer_callback
    cv_rgb = self.bridge.imgmsg_to_cv2(self.latest_rgb_msg, "bgr8")
AttributeError: 'SemanticProcessorNode' object has no attribute 'bridge'



[INFO] [1764752808.545811]: 
--- 定时触发 (50 秒)，开始语义处理 ---


[ERROR] [1764752808.546896]: 定时处理时发生错误: 'SemanticProcessorNode' object has no attribute 'bridge'
[ERROR] [1764752808.547575]: Traceback (most recent call last):
  File "/tmp/ipykernel_13511/2187352137.py", line 192, in processing_timer_callback
    cv_rgb = self.bridge.imgmsg_to_cv2(self.latest_rgb_msg, "bgr8")
AttributeError: 'SemanticProcessorNode' object has no attribute 'bridge'



In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import numpy as np
import rospy
import torch
import cv2
import struct
import json # 用于读取内参
# 注意：此节点不再需要 cv_bridge 和 message_filters

# MMSegmentation 库
from mmseg.apis import init_model, inference_model 
from mmengine.config import Config
from mmseg.registry import DATASETS 
from prettytable import PrettyTable 
from sensor_msgs.msg import PointCloud2, PointField
import sensor_msgs.point_cloud2 as pc2
from visualization_msgs.msg import Marker, MarkerArray

# --- [配置] ---
# 临时数据存储目录 (与 image_to_numpy_node.py 相同)
TEMP_DATA_DIR = "/tmp/ros_mmseg_temp"
RGB_FILE = os.path.join(TEMP_DATA_DIR, "latest_rgb.png")
DEPTH_FILE = os.path.join(TEMP_DATA_DIR, "latest_depth.npy")
INFO_FILE = os.path.join(TEMP_DATA_DIR, "latest_info.json")

FRAME_ID = 'camera_color_frame' 

# 模型和定时配置
CONFIG_FILE = '/home/mei123/workspace/cv/mmsegmentation/config/deeplabv3plus_r50-d8_4xb4-80k_ade20k-512x512.py'
CHECKPOINT_FILE = '/home/mei123/workspace/cv/mmsegmentation/config/deeplabv3plus_r50-d8_512x512_80k_ade20k_20200614_185028-bf1400d8.pth'
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
DEPTH_SCALE = 1000.0 

PROCESS_RATE_SEC = 10         # 读取文件并进行处理的触发间隔 (10 秒)
PUBLISH_RATE_HZ = 5           # 持续发布结果的频率 (5 Hz)
PUB_TOPIC_PCD_SEMANTIC = '/processed/semantic_point_cloud'
PUB_TOPIC_MARKER_ARRAY = '/processed/semantic_labels'
MARKER_LIFETIME = rospy.Duration(1.0 / PUBLISH_RATE_HZ * 1.5) 
MIN_POINTS_FOR_MARKER = 500 
MARKER_SCALE_Z = 0.1 
# ----------------------------

class SemanticProcessorNode:
    def __init__(self):
        rospy.init_node('semantic_processor_node', anonymous=True, log_level=rospy.INFO)
        
        self.pc_sem_msg = None
        self.marker_array_msg = None
        self.data_ready = False 
        
        # 依赖初始化
        self.color_cache = {} 
        self.id_to_name = self._load_model_classes()
        self.model = self._init_mmseg_model()
        
        if self.id_to_name is None or self.model is None: 
            rospy.logerr("核心依赖初始化失败，节点停止。")
            rospy.signal_shutdown("Core dependencies failed to load.")
            return

        # 发布器
        self.pub_pcd_sem = rospy.Publisher(PUB_TOPIC_PCD_SEMANTIC, PointCloud2, queue_size=1)
        self.pub_markers = rospy.Publisher(PUB_TOPIC_MARKER_ARRAY, MarkerArray, queue_size=1)

        # 启动定时器：控制处理频率 (每 10 秒读取文件并处理)
        rospy.Timer(rospy.Duration(PROCESS_RATE_SEC), self.processing_timer_callback)
        rospy.loginfo(f"处理定时器 ({PROCESS_RATE_SEC}s) 启动。")

        # 启动定时器：控制发布频率 (持续发布)
        rospy.Timer(rospy.Duration(1.0 / PUBLISH_RATE_HZ), self.publish_timer_callback)
        rospy.loginfo(f"发布定时器 ({PUBLISH_RATE_HZ} Hz) 启动。")

        self._print_color_map() 

    # --- (辅助方法，与上一个回答一致，略) ---
    def _load_model_classes(self):
        # ... (模型加载和类别解析逻辑) ...
        try:
            cfg = Config.fromfile(CONFIG_FILE)
            from mmseg.registry import DATASETS
            DatasetClass = DATASETS.get('ADE20KDataset') 
            class_names = DatasetClass.METAINFO['classes']
            return {i: name for i, name in enumerate(class_names)}
        except Exception as e:
            rospy.logerr(f"加载类别名称失败: {e}")
            return None
            
    def _init_mmseg_model(self):
        # ... (模型初始化逻辑) ...
        try:
            model = init_model(CONFIG_FILE, CHECKPOINT_FILE, device=DEVICE)
            model.eval()
            rospy.loginfo(f"MMSegmentation 模型加载成功，运行在: {DEVICE}")
            return model
        except Exception as e:
            rospy.logerr(f"模型加载失败: {e}")
            return None

    def _generate_unique_color(self, class_id: int) -> tuple:
        if class_id == 0: return (0, 0, 0)
        if class_id in self.color_cache: return self.color_cache[class_id]
        r = (class_id * 37 + 50) % 256
        g = (class_id * 101 + 100) % 256
        b = (class_id * 179 + 150) % 256
        color = (r, g, b)
        self.color_cache[class_id] = color
        return color

    def _print_color_map(self):
        table = PrettyTable()
        # ... (打印颜色映射表逻辑) ...
        table.field_names = ["ID", "Semantic Name (语义标签)", "RGB Color (点云颜色)"]
        for class_id in range(len(self.id_to_name)):
            r, g, b = self._generate_unique_color(class_id)
            class_name = self.id_to_name.get(class_id, "UNKNOWN")
            table.add_row([class_id, class_name, f"({r}, {g}, {b})"])
        rospy.loginfo("\n--- 预生成的语义点云颜色映射表 (用于 RViz 参考) ---")
        print(table)


    def _get_semantic_mask(self, cv_image: np.ndarray) -> np.ndarray:
        # 基于 cell 2 的推理逻辑
        result = inference_model(self.model, cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB))
        return result.pred_sem_seg.data.cpu().numpy().squeeze()

    def _encode_rgb_to_uint32(self, r: int, g: int, b: int) -> int:
        return (int(b) << 16) | (int(g) << 8) | int(r)

    def _create_point_cloud(self, header, data_list: list, fields):
        return pc2.create_cloud(header, fields, data_list)

    def _create_markers(self, stamp, class_centers: dict):
        # ... (MarkerArray 生成逻辑) ...
        marker_array = MarkerArray()
        marker_id_counter = 0

        for class_id, data in class_centers.items():
            if data['count'] < MIN_POINTS_FOR_MARKER: continue

            center = data['sum_xyz'] / data['count']
            class_name = self.id_to_name.get(class_id, f"ID_{class_id}")
            r, g, b = self.color_cache.get(class_id, (255, 255, 255))
            
            marker_text = Marker()
            marker_text.header.frame_id = FRAME_ID
            marker_text.header.stamp = stamp
            marker_text.ns = "semantic_labels_processed"
            marker_text.id = marker_id_counter
            marker_text.type = Marker.TEXT_VIEW_FACING
            marker_text.action = Marker.ADD
            marker_text.lifetime = MARKER_LIFETIME
            
            marker_text.pose.position.x = center[0]
            marker_text.pose.position.y = center[1]
            marker_text.pose.position.z = center[2] + MARKER_SCALE_Z 
            
            marker_text.scale.z = MARKER_SCALE_Z  
            marker_text.color.a = 1.0 
            marker_text.color.r = r / 255.0
            marker_text.color.g = g / 255.0
            marker_text.color.b = b / 255.0
            marker_text.text = class_name
            
            marker_array.markers.append(marker_text)
            marker_id_counter += 1
            
        return marker_array
    
    def _read_data_from_files(self):
        """从临时文件系统读取 RGB, 深度和内参。"""
        if not (os.path.exists(RGB_FILE) and os.path.exists(DEPTH_FILE) and os.path.exists(INFO_FILE)):
            return None, None, None
            
        try:
            # 1. 读取 RGB (cv2.imread 默认读取 BGR)
            cv_rgb = cv2.imread(RGB_FILE)
            # 2. 读取深度 (NumPy)
            cv_depth = np.load(DEPTH_FILE)
            # 3. 读取内参
            with open(INFO_FILE, 'r') as f:
                info_data = json.load(f)
                K_matrix = info_data['K']
                # 假设时间戳和 frame_id 是最新的，我们仅用 K
                fx, fy, cx, cy = K_matrix[0], K_matrix[4], K_matrix[2], K_matrix[5]
                K = [fx, fy, cx, cy]
                
            return cv_rgb, cv_depth, K
        except Exception as e:
            rospy.logerr(f"读取或解析临时文件失败: {e}")
            return None, None, None

    # --- 2. 定时处理函数 (每 10 秒触发一次) ---
    def processing_timer_callback(self, event):
        """定时获取文件数据，进行语义分割和点云生成。"""
        
        cv_rgb, cv_depth, K = self._read_data_from_files()
        
        if cv_rgb is None:
            rospy.logwarn(f"定时触发 ({PROCESS_RATE_SEC}s)，但文件数据未就绪或无法读取，跳过处理。")
            return

        rospy.loginfo(f"\n--- 定时触发，开始语义处理 (读取文件系统数据) ---")
        
        try:
            # 1. MMSegmentation 推理
            rospy.loginfo("执行 MMSegmentation 推理...")
            semantic_mask = self._get_semantic_mask(cv_rgb)
            
            # 2. 准备内参和尺寸
            fx, fy, cx, cy = K
            H, W = semantic_mask.shape

            # 3. 点云生成与融合 (仅语义点云)
            points_semantic = []
            class_centers = {} 
            
            for v in range(H):
                for u in range(W):
                    depth_mm = cv_depth[v, u]
                    if depth_mm == 0: continue
                    
                    Z = float(depth_mm) / DEPTH_SCALE
                    X = (u - cx) * Z / fx
                    Y = (v - cy) * Z / fy
                    
                    # 语义颜色
                    class_id = int(semantic_mask[v, u])
                    r_sem, g_sem, b_sem = self._generate_unique_color(class_id)
                    rgb_int_sem = self._encode_rgb_to_uint32(r_sem, g_sem, b_sem)

                    points_semantic.append((X, Y, Z, rgb_int_sem))
                    
                    # 统计 Marker 信息
                    if class_id not in class_centers:
                        class_centers[class_id] = {'sum_xyz': np.array([X, Y, Z], dtype=np.float64), 'count': 1}
                    else:
                        class_centers[class_id]['sum_xyz'] += np.array([X, Y, Z], dtype=np.float64)
                        class_centers[class_id]['count'] += 1

            # 4. 构建消息体 (存储供持续发布使用)
            pcd_fields = [
                PointField('x', 0, PointField.FLOAT32, 1), PointField('y', 4, PointField.FLOAT32, 1),
                PointField('z', 8, PointField.FLOAT32, 1), PointField('rgb', 12, PointField.UINT32, 1)
            ]
            
            # 使用当前 ROS 时间作为发布 Header 的时间戳
            header = rospy.Header()
            header.frame_id = FRAME_ID 
            
            self.pc_sem_msg = self._create_point_cloud(header, points_semantic, pcd_fields)
            self.marker_array_msg = self._create_markers(header.stamp, class_centers) 
            
            rospy.loginfo(f"新一帧语义点云和 {len(self.marker_array_msg.markers)} 个 Marker 构建完成。点数: {len(points_semantic)}")

            # 5. 设置数据可用标志位
            self.data_ready = True

        except Exception as e:
            rospy.logerr(f"定时处理时发生错误: {e}")
            import traceback
            rospy.logerr(traceback.format_exc())

    # --- 3. 持续发布函数 (高频触发) ---
    def publish_timer_callback(self, event):
        """以 PUBLISH_RATE_HZ 频率持续发布最新的处理结果。"""
        if not self.data_ready:
            return
            
        current_time = rospy.Time.now()
        
        # 更新 Header 的时间戳
        self.pc_sem_msg.header.stamp = current_time
        
        # Marker 消息需要单独更新每个 marker 的时间戳
        for marker in self.marker_array_msg.markers:
            marker.header.stamp = current_time
            
        # 1. 发布语义点云
        self.pub_pcd_sem.publish(self.pc_sem_msg)

        # 2. 发布 MarkerArray
        self.pub_markers.publish(self.marker_array_msg)
        
        rospy.logdebug(f"持续发布最新处理结果 ({current_time.to_sec():.2f})...")


if __name__ == '__main__':
    try:
        processor = SemanticProcessorNode()
        rospy.spin() 
    except rospy.ROSInterruptException:
        rospy.loginfo("ROS 中断，语义处理节点退出。")

Loads checkpoint by local backend from path: /home/mei123/workspace/cv/mmsegmentation/config/deeplabv3plus_r50-d8_512x512_80k_ade20k_20200614_185028-bf1400d8.pth
[INFO] [1764753880.442190]: MMSegmentation 模型加载成功，运行在: cuda:0
[INFO] [1764753880.444211]: 处理定时器 (10s) 启动。
[INFO] [1764753880.445070]: 发布定时器 (5 Hz) 启动。
[INFO] [1764753880.445864]: 
--- 预生成的语义点云颜色映射表 (用于 RViz 参考) ---
+-----+--------------------------+----------------------+
|  ID | Semantic Name (语义标签) | RGB Color (点云颜色) |
+-----+--------------------------+----------------------+
|  0  |           wall           |      (0, 0, 0)       |
|  1  |         building         |    (87, 201, 73)     |
|  2  |           sky            |    (124, 46, 252)    |
|  3  |          floor           |   (161, 147, 175)    |
|  4  |           tree           |    (198, 248, 98)    |
|  5  |         ceiling          |    (235, 93, 21)     |
|  6  |           road           |    (16, 194, 200)    |
|  7  |           bed            |    (53, 39, 123)